<a href="https://colab.research.google.com/github/mdfayaz21/Online-Auction-System/blob/main/Online_Auction_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask opencv-python pyngrok

In [8]:
from flask import Flask, render_template, request, redirect, url_for
from pyngrok import ngrok
import cv2
import threading

app = Flask(__name__)

# Dummy data for auction items
auction_items = [
    {"id": 1, "name": "Painting", "current_bid": 100},
    {"id": 2, "name": "Antique Vase", "current_bid": 200},
    {"id": 3, "name": "Vintage Watch", "current_bid": 150},
]

# Home page
@app.route('/')
def home():
    return render_template('home.html', items=auction_items)

# Bid on an item
@app.route('/bid/<int:item_id>', methods=['GET', 'POST'])
def bid(item_id):
    item = next((item for item in auction_items if item["id"] == item_id), None)
    if request.method == 'POST':
        bid_amount = int(request.form['bid_amount'])
        if bid_amount > item['current_bid']:
            item['current_bid'] = bid_amount
            return redirect(url_for('home'))
    return render_template('bid.html', item=item)

# Real-time face detection page
@app.route('/face_detection')
def face_detection():
    return render_template('face_detection.html')

# Run the Flask app
def run_flask():
    app.run(host='0.0.0.0', port=5001)

# Start Flask in a separate thread
threading.Thread(target=run_flask).start()

# Set your Ngrok authtoken
ngrok.set_auth_token("2sS9VDhL61GUf2mNF1wxkRfBIim_3wnXjJd6dgC78eYXyEEPh")  # Replace with your Ngrok authtoken

# Expose the app using ngrok
public_url = ngrok.connect(5000)
print("Auction System is live at:", public_url)

 * Serving Flask app '__main__'
 * Debug mode: off
Auction System is live at: NgrokTunnel: "https://e70c-34-141-172-195.ngrok-free.app" -> "http://localhost:5000"


Address already in use
Port 5001 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [7]:
from flask import Response
import cv2

# Load the Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Video feed generator
def generate_frames():
    cap = cv2.VideoCapture(0)
    while True:
        success, frame = cap.read()
        if not success:
            break
        else:
            # Convert frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Detect faces
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            # Draw rectangles around detected faces
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            # Encode the frame as JPEG
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

# Video feed route
@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')